In [1]:
import json, tensorflow as tf, pandas as pd, numpy as np
import tensorflow_hub as hub
#model from https://www.tensorflow.org/tutorials/keras/text_classification_with_hub

In [2]:
with open("./scryfall-oracle-cards.json", "r", encoding="utf-8") as file:
    fileobj=json.loads(file.read())

In [3]:
dataset=[]
labels=[]

for card in fileobj:
    if card["layout"]!="normal" or len(card["colors"])!=1:
        continue
    dataset.append([card["name"], str(card["cmc"]), card["type_line"], card["oracle_text"]])
    labels.append(card["colors"][0])
fileobj=None


In [4]:
splitpoint=int(len(dataset)*.75)
print("Split point is:", splitpoint)
train_examples=np.asarray(dataset[:splitpoint])
train_labels=np.asarray(labels[:splitpoint])
test_examples=np.asarray(dataset[splitpoint:])
test_labels=np.asarray(labels[splitpoint:])
print(len(train_examples))
print(len(train_labels))

Split point is: 11296
11296
11296


In [5]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_examples, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_examples, test_labels))

In [6]:
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [7]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)

In [8]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [10]:
history = model.fit(train_dataset.shuffle(10000).batch(512),
                    epochs=20,
                    verbose=1)


ValueError: in converted code:

    c:\users\sally\documents\2020\tfpractice\env\lib\site-packages\tensorflow_core\python\keras\engine\training_v2.py:677 map_fn
        batch_size=None)
    c:\users\sally\documents\2020\tfpractice\env\lib\site-packages\tensorflow_core\python\keras\engine\training.py:2410 _standardize_tensors
        exception_prefix='input')
    c:\users\sally\documents\2020\tfpractice\env\lib\site-packages\tensorflow_core\python\keras\engine\training_utils.py:573 standardize_input_data
        'with shape ' + str(data_shape))

    ValueError: Error when checking input: expected keras_layer_input to have 1 dimensions, but got array with shape (None, None, 4)
